In [ ]:
from labda import Subject
from labda.core.utils import get_missing_records
from labda.spatial.impute import impute_gaps

sbj = Subject.from_parquet("78_carlson.parquet")
sf = sbj.metadata.sampling_frequency
crs = sbj.metadata.crs
timezone = sbj.metadata.timezone

# sbj.df = sbj.df[["latitude", "longitude"]]
# sbj.set_timeframe(start="2011-07-14 08:45:00", end="2011-07-14 09:15:00")

distance = 100
velocity = 1 * 3.6
duration = "5m"

df = sbj.df
nsat_used = (df[df["nsat_used"] < 4]).index
df.drop(nsat_used, inplace=True)
# df.drop(index=df.iloc[60:90].index, inplace=True)
missing_records = get_missing_records(df, sf)

if missing_records.empty:
    raise ValueError("No missing records found.")


df, gaps, fuzzy = impute_gaps(df, crs, sf, duration, duration, velocity, distance)
fuzzy

1 m/s, 100 m - STOP if less than that.
IMPUTED if both gap distance and velocity large

large gap distance, small velocity - SAM (stop-and-move)
small gap, large velocity - SOM (stop or move)

MV - gap velocity (1 m/s)
SR - gap distance (100 m)

Any gaps ranging from 90 sec to 2 days (tracking period) are imputed.

imputed only if between MinStopDuration and MaxStopDuration

#####

HDOP >= 4
low number of satellites used < 4
no location fix
redudant records small distance < 10 or less (depending on interval)

> 45 m/s removed,
abrupt location change - a location deviation score is calculated as a ratio of the distance between the current point p and the representative location of p’s temporal neighbor to standard distance of p’s temporal neighbor. The higher the location deviation score, the more the current location is deviated from its temporal neighbor. Data points with a location deviation score >2.4 are marked as outliers based on experimentations.

fuzzy - 

In [ ]:
import pandas as pd

from labda.core.resample import resample
from labda.spatial.expanders import get_distance, get_speed

df = pd.read_parquet("../temp/actigraph_qstarz_testing_files/test.parquet")
metadata = df.attrs
crs = metadata["crs"]

df = resample(df, "15s")
df["speed"] = get_speed(df)
df["distance"] = get_distance(df, crs)
# all_nan_rows = df[df.isna().all(axis=1)]
# all_nan_rows

In [ ]:
df["info"] = "valid"
df.loc[(df["speed"] > 200), "info"] = "high_speed"
df.loc[(df["snr_used"] < 100), "info"] = "low_snr"
df.loc[(df["sat_ratio"] < 0.5), "info"] = "low_sat_ratio"
df.loc[df["sat_used"] < 4, "info"] = "fewer satellites"
df["info"].value_counts()